In [80]:
import os
import sys
import pandas as pd
import glob
import shutil
import numpy as np

base_dir = os.path.realpath('D:\\sky_dataset')

In [69]:
#all_img_df = pd.DataFrame(dict(path = glob.glob(os.path.join(base_dir,'sky_image','10066', '*.jpg*'))))
path = []
for num in sorted(os.listdir(os.path.join(base_dir,'sky_image'))):
    path += glob.glob(os.path.join(base_dir,'sky_image',num, '*.jpg*'))
all_img_df = pd.DataFrame(dict(path = path))


In [106]:
import re
counter = 0
mask_path = []
id = [] 
for x in path:
    num = (re.findall('\d+', m_p ))
    id.append(num[0])
    mask_path += glob.glob(os.path.join(base_dir, 'sky_mask', str(num[0]) + '.png')) 
all_img_df["sky_mask"] = mask_path
all_img_df["id"] = id


In [112]:
all_img_df["key_id"] = all_img_df['path'].map(lambda x : splitext(os.path.basename(x))[0])
all_img_df['sky_id'] = all_img_df['key_id'].map(lambda x: x.split('_')[1])
all_img_df.head()
from sklearn.model_selection import train_test_split
def train_test_split_on_group()

,path,sky_mask,id,key_id,sky_id
0,D:\sky_dataset\sky_image\10066\20130101_084634...,D:\sky_dataset\sky_mask\10066.png,10066,20130101_084634,084634
1,D:\sky_dataset\sky_image\10066\20130101_091628...,D:\sky_dataset\sky_mask\10066.png,10066,20130101_091628,091628
2,D:\sky_dataset\sky_image\10066\20130101_114638...,D:\sky_dataset\sky_mask\10066.png,10066,20130101_114638,114638
3,D:\sky_dataset\sky_image\10066\20130101_125725...,D:\sky_dataset\sky_mask\10066.png,10066,20130101_125725,125725
4,D:\sky_dataset\sky_image\10066\20130101_154632...,D:\sky_dataset\sky_mask\10066.png,10066,20130101_154632,154632


20130101_084634
